In [2]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as c
from matplotlib import rcParams
import sympy as sy
import scipy.constants as pc
from sympy import *
from math import log10, floor

In [3]:
# Funktionen
def am(werte):
    return np.mean(werte)

def std(werte):
    summen_der_quadrate = 0
    for wert in werte:
        summen_der_quadrate+=(wert-am(werte))**2       
    return np.sqrt(summen_der_quadrate/(werte.size-1))

def stu(werte):
    return std(werte)/np.sqrt(werte.size)

x = "Johann ist der beste"

def fehler(f, werte, unsicherheiten, variablen):
    zMax = 0
    for i in range(werte.size):
        dif = np.abs(sy.diff(f, variablen[i]))
        zMax += (dif * unsicherheiten[i])**2
    zMax = sqrt(zMax)
    for j in range(werte.size):
        zMax = sub(zMax, variablen[j], werte[j])
    return zMax

def fehlerMax(f, werte, unsicherheiten, variablen):
    zMax = 0
    for i in range(werte.size):
        dif = np.abs(sy.diff(f, variablen[i]))
        zMax += dif * unsicherheiten[i]
    for j in range(werte.size):
        zMax = sub(zMax, variablen[j], werte[j])
    return zMax

def groessterBeitrag(f, werte, unsicherheiten, variablen):
    zMax = 0
    zMaxArray = np.array([])
    for i in range (werte.size):
        zMax = (np.abs(sy.diff(f, variablen[i])) * unsicherheiten[i])
        for j in range(werte.size):
            zMax = sub(zMax, variablen[j], werte[j])
        x = np.array([zMax])
        zMaxArray = np.append(zMaxArray, x)
    return variablen[np.argmax(zMaxArray)]

def sub(zMax, variable, wert):
    return zMax.subs([(variable, wert)])

def einfachGaussAdd(unsicherheiten, koeffizienten):
    dz = 0
    for i in range (unsicherheiten.size):
        dz += (unsicherheiten[i]*unsicherheiten[i])**2
    return sqrt(dz)

def einfachGaussMult(unsicherheiten, werte, exponenten):
    dz = 0
    for i in range (unsicherheiten.size):
        dz += (exponenten[i]*unsicherheiten[i]/werte[i])**2
    return sqrt(dz)

def einfachGaussMultMax(unsicherheiten, werte, exponenten):
    max_a = 0
    num = -1
    for i in range (unsicherheiten.size):
        maxi = (exponenten[i]*unsicherheiten[i]/werte[i])**2
        if max_a <= maxi:
            max_a = maxi
            num = i
    return (max_a, num)

def vertraeglichkeit(am_1, am_2, stu_1, stu_2):
    return (np.abs(am_1-am_2)/(sqrt((stu_1**2)*(stu_2**2))))

def bestA(x, y):
    return (sum(x**2)*sum(y)-sum(x)*sum(x*y))/(x.size*sum(x**2)-sum(x)**2)

def bestB(x, y):
    return (x.size*sum(x*y)-sum(x)*sum(y))/(x.size*sum(x**2)-sum(x)**2)

def streuung(x, y):
    s = 0
    for i in range(x.size):
        s += (y[i]-(bestA(x, y)+bestB(x, y)*x[i]))**2
    return np.sqrt(s/(x.size-2))

def stuA(x, y):
    s = streuung(x, y)
    return s*np.sqrt(sum(x**2)/((x.size*sum(x**2)-sum(x)**2)))

def stuB(x, y):
    s = streuung(x, y)
    return s*np.sqrt(x.size/((x.size*sum(x**2)-sum(x)**2)))

def round_to_j(x,j):
    return round(x, (j-1)-int(floor(log10(abs(x)))))

In [4]:
# Erinnerungen
# x = np.array([,])
# x.size
# x, y = symbols('x y')
# expr = x*y
# expr.subs([(x, wert_x), (y, wert_y)])

In [5]:
# Einzelmessungen
gew = np.array([0, 10, 20, 30, 40, 50])
f_gew = 0.1
auslenk =  abs(np.array([365, 311, 259, 208, 150, 103])-365)
f_auslenk = 1

In [6]:
klaengeg = np.linspace(0, 800, 9)+107+5
omega_g = 2*pc.pi/(np.array([19.02, 18.71, 18.74, 18.74, 18.91, 18.87, 18.75, 18.73, 18.77])/10)
us_p = am(2*pc.pi/(np.array([19.02, 18.71, 18.74, 18.74, 18.91, 18.87, 18.75, 18.73, 18.77])/10))*am(0.05/(np.array([19.02, 18.71, 18.74, 18.74, 18.91, 18.87, 18.75, 18.73, 18.77])/10))
#d = np.array([3.2, 3.9, 6.3, 2.75, 4.9, 4.3, 4.3, 3.3, 3.9])
pTe = np.array([18.7, 18.41, 18.03, 17.80, 17.11, 16.49, 16.05, 15.27, 14.76])/10

In [7]:
#Kopplungs
pTp = 2*pc.pi/(np.array([14.86/10, 15.39/10, 16.45/10, 16.79/10, 18.04/10, 16.22/9, 16.66/9, 16.79/9, 17.08/9]))
pTm = 2*pc.pi/(np.array([12.3 * 0.5, 15.89 * 0.5, 21.20 * 0.5, 28.17*0.5, 18.94, 26.86, 23.45 * 2, 49.71 * 2, 76.78 * 4])) #in einer Periode

zu_rundende_arrays = np.array([pTp, pTm])

for i in range(len(zu_rundende_arrays)):
    
    runden_array = zu_rundende_arrays[i]
    for j in range(len(runden_array)):
        
        runden_array[j] = round_to_j(runden_array[j],3)
    runden_array = zu_rundende_arrays[i]

for i in range(len(zu_rundende_arrays)):
    print(i+1,zu_rundende_arrays[i])
        


1 [4.23 4.08 3.82 3.74 3.48 3.49 3.39 3.37 3.31]
2 [1.02   0.791  0.593  0.446  0.332  0.234  0.134  0.0632 0.0205]


In [41]:
#Fehlerfortpüflanzung g,f

g_w, g_f = 9.808, 0.001
l_w, l_f = 0.88, 0.01 #Stimmt nicht YEP monkaSTEER ඞඞඞඞඞඞ AMඞGUS, WENN DER WERT SUS IST ඞඞඞඞඞඞඞඞ
omega_lit_w = np.sqrt(g_w/l_w)

print("OMEGALUL", np.sqrt(g_w/l_w)) #3.34 gemessen, Ungenauigkeit 0,09

delta_omega_f = einfachGaussMult(np.array([g_f, l_f]), np.array([g_w, l_w]), np.array([0.5,-0.5]))
omega_lit_f = delta_omega_f * omega_lit_w
print("Fehler relativ:", delta_omega_f)
print("Fehler total:", omega_lit_f)

ver_omega = vertraeglichkeit(omega_lit_w, 3.34, omega_lit_f, 0.09)
print(ver_omega)

OMEGALUL 3.338480873908752
Fehler relativ: 0.00568204687489969
Fehler total: 0.0189694048165056
0.889810669070355
